Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [4]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import regex as r

In [21]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [89]:
print(posts[7])

<article class="post post_preview" lang="ru">
<header class="post__meta">
<a class="post__user-info user-info" href="https://habr.com/ru/users/fillpackart/" title="Автор публикации">
<img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/7eb/b30/613/7ebb306137a00d2f53bbef9235f351c5.jpg" width="24"/>
<span class="user-info__nickname user-info__nickname_small">fillpackart</span>
</a>
<span class="post__time">сегодня в 18:30</span>
</header>
<h2 class="post__title">
<a class="post__title_link" href="https://habr.com/ru/company/vdsina/blog/523618/">Я принес вам решение проблемы с исключениями в C#. Но вам не понравится</a>
</h2>
<ul class="post__hubs inline-list">
<li class="inline-list__item inline-list__item_hub">
<a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/vdsina/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании VDSina.ru'); }" rel="nofollow" tit

In [104]:
KEYWORDS = ['парсинг','исключения','CSS']

In [105]:
search_post = pd.DataFrame()
for i in posts:
    for j in KEYWORDS:
        if i.find(string=r.compile(j)):
            date = i.find(class_='post__time').text
            if 'сегодня' in date:
                tmp_d = str(datetime.date(datetime.now()))
                tmp_t = r.findall(r'\d{2}:\d{2}', date)
                date = tmp_d + ' ' + str(tmp_t[0])
            title = i.find('h2', class_='post__title').text
            href = i.find('a', class_='post__title_link').get('href')
            row = {'date': date, 'title': title, 'href': href}
            search_post = search_post.append(pd.DataFrame([row])) 
        

In [106]:
search_post.head()

,date,title,href
0,2020-10-21 19:38,\nКак я структурирую CSS\n,https://habr.com/ru/post/523884/
0,2020-10-21 18:30,\nЯ принес вам решение проблемы с исключениями...,https://habr.com/ru/company/vdsina/blog/523618/


Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [153]:
KEYWORDS = ['парсинг','python']

In [154]:
search_post = pd.DataFrame()
for i in posts:
    for j in KEYWORDS:
        if i.find(string=r.compile(j)):
            date = i.find(class_='post__time').text
            if 'сегодня' in date:
                tmp_d = str(datetime.date(datetime.now()))
                tmp_t = r.findall(r'\d{2}:\d{2}', date)
                date = tmp_d + ' ' + str(tmp_t[0])
            title = i.find('h2', class_='post__title').text
            href = i.find('a', class_='post__title_link').get('href')
            tmp_h = i.find('a', class_='post__title_link').get('href')
            req_2 = requests.get(tmp_h)
            soup_2 = BeautifulSoup(req_2.text, 'html.parser')
            text = soup_2.find_all(id="post-content-body")
            row = {'date': date, 'title': title, 'href': href, 'text': text}
            search_post = search_post.append(pd.DataFrame([row])) 
        else:
            tdate = i.find(class_='post__time').text
            if 'сегодня' in date:
                tmp_d = str(datetime.date(datetime.now()))
                tmp_t = r.findall(r'\d{2}:\d{2}', date)
                date = tmp_d + ' ' + str(tmp_t[0])
            title = i.find('h2', class_='post__title').text
            href = i.find('a', class_='post__title_link').get('href')
            tmp_h = i.find('a', class_='post__title_link').get('href')
            req_2 = requests.get(tmp_h)
            soup_2 = BeautifulSoup(req_2.text, 'html.parser')
            text = soup_2.find_all(id="post-content-body")
            if soup_2.find(string=r.compile(j)):
                row = {'date': date, 'title': title, 'href': href, 'text': text}
                search_post = search_post.append(pd.DataFrame([row]))

In [155]:
search_post.head()

,date,title,href,text
0,2020-10-21 18:30,\nСпособы и примеры внедрения утилит для прове...,https://habr.com/ru/company/swordfish_security...,"[[[], [], \r\nПривет, Хабр!, [], \n, [], \r\nВ..."
0,2020-10-21 18:30,\nУмное зеркало своими руками\n,https://habr.com/ru/company/ruvds/blog/524166/,"[[Кому нужно умное зеркало? Много кому. Я, ког..."
0,2020-10-21 18:30,\nКак стать кросс-функциональной командой\n,https://habr.com/ru/company/oleg-bunin/blog/34...,[[DevOps обычно рассматривается в двух ипостас...


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [1]:
import json

In [5]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

header = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
}

In [6]:
EMAIL = ['xxx@x.ru', 'yyy@y.com' , 'davidoff_max_min@ya.ru']

In [17]:
search_email = pd.DataFrame()
for i in EMAIL:
    params = {
        'emailAddresses' : [i]
    }
    res = requests.post(URL, data = json.dumps(params), headers = header)
    tmp = json.loads(res.text)
    for j,k in tmp.items():
        if j == 'breaches':
            for l in k.values():
                tmp_date = l['publishDate']
                row_2 = {'email': i, 'date': tmp_date[:10], 'source': l['site'], 'description': l['description']}
                search_email = search_email.append(pd.DataFrame([row_2]))



In [18]:
search_email.head(10)

,email,date,source,description
0,xxx@x.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-02-14,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,2020-01-03,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2020-05-28,wishbone.io,"In January 2020, the online poll website Wishb..."
0,yyy@y.com,2017-11-04,myheritage.com,"In October 2017, a customer database belonging..."
0,yyy@y.com,2019-06-13,canva.com,"In May 2019, graphic-design site Canva's datab..."
